In [1]:
import numpy as np
import pandas as pd
from pulp import *
from ortoolpy import addvars, addbinvars
import random

In [2]:
# ペナルティの重み定数
C_need_dif = 1000
C_lock = 50
C_3continuous_work = 20
C_2continuous_work = 3
C_average = 10
C_gender = 3
C_experience = 3
C_blank = 1

In [3]:
# ここでExcelのデータを取得
shift = pd.read_csv('data/Shift.tsv', sep='\t',  index_col=0)
manage = pd.read_csv('data/Manage.tsv', sep='\t', index_col=0)
member = pd.read_csv('data/Member.tsv', sep='\t', index_col=0).T
detail = pd.read_csv('data/Detail.tsv', sep='\t', index_col=0)

In [4]:
shift.head()

,Aさん,Bさん,Cさん,Dさん,Eさん,Fさん,Gさん,Hさん,Iさん,Jさん,Kさん,Lさん,Mさん
1,1,0,1,0,0,0,0,1,0,1,1,1,1
2,1,0,1,0,1,0,0,1,0,1,0,1,1
4,1,1,0,0,1,0,0,1,0,1,0,0,0
5,1,0,0,1,1,0,0,0,0,1,0,1,1
6,1,1,0,1,1,0,0,0,0,1,1,1,1


In [5]:
manage.head()

,need
1,5
2,4
4,4
5,4
6,4


In [6]:
member.head()

,amount,sex,rank
Aさん,1,1,1
Bさん,1,1,1
Cさん,4,1,1
Dさん,1,0,1
Eさん,1,0,1


In [7]:
detail.head()

,more,normal,less,zero,rock
max,0,8,0,0,0
min,0,7,0,0,0


In [8]:
# 確認用にコピー
shift_show = shift.copy()

In [9]:
# Shiftシートのカラムとインデックス
date = shift.index
employee = shift.columns

In [10]:
date

Int64Index([1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
            21, 22, 23],
           dtype='int64')

In [11]:
employee

Index(['Aさん', 'Bさん', 'Cさん', 'Dさん', 'Eさん', 'Fさん', 'Gさん', 'Hさん', 'Iさん', 'Jさん',
       'Kさん', 'Lさん', 'Mさん'],
      dtype='object')

In [12]:
# 従業員数と月の日数
days, number_employee = shift.shape[0], shift.shape[1]
days, number_employee

(22, 13)

In [13]:
# 変数作成
var = pd.DataFrame(np.array(addbinvars(days, number_employee)), columns=employee, index=date)
# ０と１を逆転させている
shift_rev = shift[shift.columns].apply(lambda x: 1 - x[shift.columns], axis=1)

In [14]:
var.head()

,Aさん,Bさん,Cさん,Dさん,Eさん,Fさん,Gさん,Hさん,Iさん,Jさん,Kさん,Lさん,Mさん
1,v000001,v000002,v000003,v000004,v000005,v000006,v000007,v000008,v000009,v000010,v000011,v000012,v000013
2,v000014,v000015,v000016,v000017,v000018,v000019,v000020,v000021,v000022,v000023,v000024,v000025,v000026
4,v000027,v000028,v000029,v000030,v000031,v000032,v000033,v000034,v000035,v000036,v000037,v000038,v000039
5,v000040,v000041,v000042,v000043,v000044,v000045,v000046,v000047,v000048,v000049,v000050,v000051,v000052
6,v000053,v000054,v000055,v000056,v000057,v000058,v000059,v000060,v000061,v000062,v000063,v000064,v000065


In [15]:
# 入れる場所が0、入れない場所が1を表す
shift_rev.head()

,Aさん,Bさん,Cさん,Dさん,Eさん,Fさん,Gさん,Hさん,Iさん,Jさん,Kさん,Lさん,Mさん
1,0,1,0,1,1,1,1,0,1,0,0,0,0
2,0,1,0,1,0,1,1,0,1,0,1,0,0
4,0,0,1,1,0,1,1,0,1,0,1,1,1
5,0,1,1,0,0,1,1,1,1,0,1,0,0
6,0,0,1,0,0,1,1,1,1,0,0,0,0


In [17]:
k = LpProblem('shift', sense=LpMinimize)

In [18]:
# 希望していない場所には入らないようにする。
for (_, h), (_, n) in zip(shift_rev.iterrows(), var.iterrows()):
    k += lpDot(h, n) <= 0

In [19]:
# 変数の追加
shift['V_need_dif'] = addvars(days)  # 必要人数に達していない時のペナルティ変数
shift['V_gender_rate'] = addvars(days)  # 女性が少ない時のペナルティ
shift['V_experience'] = addvars(days)  # 新人だけになった時のペナルティ
shift['need'] = manage.need  # 必要人数

In [20]:
shift.head()

,Aさん,Bさん,Cさん,Dさん,Eさん,Fさん,Gさん,Hさん,Iさん,Jさん,Kさん,Lさん,Mさん,V_need_dif,V_gender_rate,V_experience,need
1,1,0,1,0,0,0,0,1,0,1,1,1,1,v000287,v000309,v000331,5
2,1,0,1,0,1,0,0,1,0,1,0,1,1,v000288,v000310,v000332,4
4,1,1,0,0,1,0,0,1,0,1,0,0,0,v000289,v000311,v000333,4
5,1,0,0,1,1,0,0,0,0,1,0,1,1,v000290,v000312,v000334,4
6,1,1,0,1,1,0,0,0,0,1,1,1,1,v000291,v000313,v000335,4


In [21]:
V_3continuous_work = np.array(addbinvars(days - 2, number_employee)) # 3日連続勤務
V_2continuous_work = np.array(addbinvars(days - 1, number_employee)) # 2日連続勤務
V_blank = np.array(addbinvars(days - 6, number_employee))
V_max = addvars(number_employee)
V_min = addvars(number_employee)
V_lock = addvars(number_employee)

In [22]:
# 足りない人がいれば終了
shortage = []
for index, r in shift[employee].iterrows():
    if sum(r) < int(shift.at[index, 'need']):
        shortage.append(index)
if shortage:
    day = ''
    for i in shortage:
        day += (str(i) + '日足りません')
        print(day)
#     exit()

In [23]:
# 必要な人数に対するペナルティーを求める。
for (_, s), (_, v) in zip(shift.iterrows(), var.iterrows()):
    k += s.V_need_dif >= (lpSum(v) - s.need) # 多すぎるとペナルティ
    k += s.V_need_dif >= -(lpSum(v) - s.need) # 少なすぎるとペナルティ

In [24]:
# 連勤に対してペナルティーを求める。
for i in list(range(number_employee)):
    for n, p in enumerate((var.values[:-2, i] + var.values[1:-1, i] + var.values[2:, i]).flat):
#     for n, p in enumerate(var.values[:-2, i] + var.values[1:-1, i] + var.values[2:, i]):
        k += p - V_3continuous_work[n][i] <= 2

In [25]:
for i in list(range(number_employee)):
    for n, p in enumerate(var.values[:-1, i] + var.values[1:, i]):
        k += p - V_2continuous_work[n][i] <= 1

In [26]:
# 長く空きすぎるとペナルティ
for i in list(range(number_employee)):
    for n, p in enumerate(var.values[:-6, i] + var.values[1:-5, i] + var.values[2:-4, i] + var.values[3:-3, i] + var.values[4:-2, i] + var.values[5:-1, i] + var.values[6:, i]):
        k += p + V_blank[n][i] >= 1

In [27]:
# ある程度の入る量を調整
amount_more_user_list = member[member['amount'].isin([0])].index
amount_normal_user_list = member[member['amount'].isin([1])].index
amount_less_user_list = member[member['amount'].isin([2])].index
amount_zero_user_list = member[member['amount'].isin([3])].index
amount_lock_user_list = member[member['amount'].isin([4])].index

In [28]:
# 入りすぎ、入らなすぎにペナルティを与える変数
V_shift_max = pd.DataFrame(V_max, index=employee).T
V_shift_min = pd.DataFrame(V_min, index=employee).T
V_shift_lock = pd.DataFrame(V_lock, index=employee).T

In [37]:
V_shift_max

,Aさん,Bさん,Cさん,Dさん,Eさん,Fさん,Gさん,Hさん,Iさん,Jさん,Kさん,Lさん,Mさん
0,v001094,v001095,v001096,v001097,v001098,v001099,v001100,v001101,v001102,v001103,v001104,v001105,v001106


In [29]:
# Excelから取得したデータをもとに
amount_more_setting = {'max': detail.at['max', 'more'], 'min': detail.at['min', 'more']}
amount_normal_setting = {'max': detail.at['max', 'normal'], 'min': detail.at['min', 'normal']}
amount_less_setting = {'max': detail.at['max', 'less'], 'min': detail.at['min', 'less']}

In [39]:
amount_normal_setting

{'max': 8, 'min': 7}

In [40]:
var[amount_normal_user_list]

,Aさん,Bさん,Dさん,Eさん,Fさん,Gさん,Hさん,Iさん,Jさん,Kさん,Lさん,Mさん
1,v000001,v000002,v000004,v000005,v000006,v000007,v000008,v000009,v000010,v000011,v000012,v000013
2,v000014,v000015,v000017,v000018,v000019,v000020,v000021,v000022,v000023,v000024,v000025,v000026
4,v000027,v000028,v000030,v000031,v000032,v000033,v000034,v000035,v000036,v000037,v000038,v000039
5,v000040,v000041,v000043,v000044,v000045,v000046,v000047,v000048,v000049,v000050,v000051,v000052
6,v000053,v000054,v000056,v000057,v000058,v000059,v000060,v000061,v000062,v000063,v000064,v000065
7,v000066,v000067,v000069,v000070,v000071,v000072,v000073,v000074,v000075,v000076,v000077,v000078
8,v000079,v000080,v000082,v000083,v000084,v000085,v000086,v000087,v000088,v000089,v000090,v000091
9,v000092,v000093,v000095,v000096,v000097,v000098,v000099,v000100,v000101,v000102,v000103,v000104
10,v000105,v000106,v000108,v000109,v000110,v000111,v000112,v000113,v000114,v000115,v000116,v000117
11,v000118,v000119,v000121,v000122,v000123,v000124,v000125,v000126,v000127,v000128,v000129,v000130


In [30]:
# 制約
for name, r in var[amount_more_user_list].iteritems():
    k += lpSum(r) + V_shift_max.at[0, name] >= amount_more_setting['min']
    k += lpSum(r) - V_shift_min.at[0, name] <= amount_more_setting['max']
for name, r in var[amount_normal_user_list].iteritems():
    k += lpSum(r) + V_shift_max.at[0, name] >= amount_normal_setting['min']
    k += lpSum(r) - V_shift_min.at[0, name] <= amount_normal_setting['max']
for name, r in var[amount_less_user_list].iteritems():
    k += lpSum(r) + V_shift_max.at[0, name] >= amount_less_setting['min']
    k += lpSum(r) - V_shift_min.at[0, name] <= amount_less_setting['max']
for (_, h), (name, n) in zip(shift[amount_lock_user_list].iteritems(), var[amount_lock_user_list].iteritems()):
    k += lpSum(n) + V_shift_lock.at[0, name] >= lpSum(h)
    k += lpSum(n) - V_shift_lock.at[0, name] <= lpSum(h)

In [31]:
# 男女偏り、新人のみにならない
woman_list = member[member['sex'].isin([1])].index
veteran_list = member[member['rank'].isin([1])].index
for (_, r), (_, d) in zip(shift.iterrows(), var[woman_list].iterrows()):
    if r.need == 0:
        pass
    else:
        k += (r.V_gender_rate + lpSum(d)) >= 2
for (_, r), (_, d) in zip(shift.iterrows(), var[veteran_list].iterrows()):
    if r.need == 0:
        pass
    else:
        k += (r.V_experience + lpSum(d)) >= 1

In [32]:
# 目的関数決定
k += C_need_dif * lpSum(shift.V_need_dif) \
     + C_3continuous_work * lpSum(V_3continuous_work) \
     + C_2continuous_work * lpSum(V_2continuous_work) \
     + C_average * lpSum(V_max) \
     + C_average * lpSum(V_min) \
     + C_gender * lpSum(shift.V_gender_rate) \
     + C_experience * lpSum(shift.V_experience) \
     + C_lock * lpSum(V_lock) \
     + C_blank * lpSum(V_blank)

In [33]:
print(k)

shift:
MINIMIZE
1000*v000287 + 1000*v000288 + 1000*v000289 + 1000*v000290 + 1000*v000291 + 1000*v000292 + 1000*v000293 + 1000*v000294 + 1000*v000295 + 1000*v000296 + 1000*v000297 + 1000*v000298 + 1000*v000299 + 1000*v000300 + 1000*v000301 + 1000*v000302 + 1000*v000303 + 1000*v000304 + 1000*v000305 + 1000*v000306 + 1000*v000307 + 1000*v000308 + 3*v000309 + 3*v000310 + 3*v000311 + 3*v000312 + 3*v000313 + 3*v000314 + 3*v000315 + 3*v000316 + 3*v000317 + 3*v000318 + 3*v000319 + 3*v000320 + 3*v000321 + 3*v000322 + 3*v000323 + 3*v000324 + 3*v000325 + 3*v000326 + 3*v000327 + 3*v000328 + 3*v000329 + 3*v000330 + 3*v000331 + 3*v000332 + 3*v000333 + 3*v000334 + 3*v000335 + 3*v000336 + 3*v000337 + 3*v000338 + 3*v000339 + 3*v000340 + 3*v000341 + 3*v000342 + 3*v000343 + 3*v000344 + 3*v000345 + 3*v000346 + 3*v000347 + 3*v000348 + 3*v000349 + 3*v000350 + 3*v000351 + 3*v000352 + 20*v000353 + 20*v000354 + 20*v000355 + 20*v000356 + 20*v000357 + 20*v000358 + 20*v000359 + 20*v000360 + 20*v000361 + 20*v00036

In [77]:
k.solve()

1

In [78]:
result = np.vectorize(value)(var).astype(int)
R_continuous_work = np.vectorize(value)(V_2continuous_work).astype(int)
print('目的関数', value(k.objective))
print(result)

目的関数 103.0
[[1 0 1 0 0 0 0 1 0 0 1 0 1]
 [0 0 1 0 1 0 0 0 0 0 0 1 1]
 [1 1 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 1 1]
 [1 0 1 0 0 0 0 1 1 0 1 0 0]
 [0 0 1 1 1 0 0 1 0 0 1 0 0]
 [0 1 0 0 0 0 1 0 1 0 0 1 1]
 [0 0 0 1 0 1 1 1 0 1 0 0 0]
 [0 1 0 0 1 0 0 0 1 0 0 1 1]
 [0 0 0 1 0 1 1 1 0 1 0 0 0]
 [1 1 0 1 0 1 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 1 1 1 1 0 0 0]
 [0 1 0 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 0 1 1 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 0 1 1 0 0 1 0 0 0]
 [0 0 0 1 1 1 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 1 0 0]
 [0 0 0 1 1 0 0 0 0 0 1 1 0]]


In [80]:
continuous_work_list = []
for i in np.sum(R_continuous_work, axis=0):
    if i >= 1:
        continuous_work_list.append('有')
    else:
        continuous_work_list.append('無')

fi = []
for cou, r in enumerate(result):
    fi.append([])
    for i, j in zip(r, shift.columns):
        if i * j != '':
            fi[cou].append(i * j)

amount = []
member_rev = member.T
for name, r in member_rev.iteritems():
    if r.amount == 0:
        amount.append("多め")
    elif r.amount == 1:
        amount.append("普通")
    elif r.amount == 2:
        amount.append("少なめ")
    elif r.amount == 3:
        amount.append("無し")
    elif r.amount == 4:
        amount.append("固定")

count = np.sum(result, axis=0)
evaluation = pd.DataFrame([amount, count, continuous_work_list], index=['シフト希望量', '入る量', '連勤'], columns=employee)
print(evaluation)

       Aさん Bさん Cさん   D   E   F   G   H   I   J   K   L   M
シフト希望量  普通  普通  固定  普通  普通  普通  普通  普通  普通  普通  普通  普通  普通
入る量      9   9   4   9   8   7   8   7   8   8   8   8   6
連勤       有   無   有   有   無   有   有   有   有   無   有   無   有


In [82]:
# ここで目視用のシートを作る
result_show_color = pd.DataFrame(result, index=date, columns=employee)
for index, i in result_show_color.iterrows():
    for column, c in i.iteritems():
        if c == 1:  # 実際入ってる人
            pass
        elif c == 0 and shift.at[index, column] == 1:  # 入る希望はあったが入らない人
            pass
        else:
            result_show_color.at[index, column] = 2  # 入るつもりがない人

shift_member = fi
for values in shift_member:
    random.shuffle(values)
shift_result = pd.DataFrame(shift_member, index=date)

shift_result.to_csv('result/result.tsv', sep='\t')
evaluation.to_csv('result/eval.tsv', sep='\t')
result_show_color.to_csv('result/show.tsv', sep='\t')